# CMB probes in the Euclid likelihood

We show how to obtain the CMB observables including:
- cross correlation between CMB lensing convergence and galaxy weak lensing 
- cross correlation between CMB lensing convergence and galaxy clustering photometric
- auto correlation of CMB lensing convergence
- cross correlation between CMB temperature and galaxy clustering photometric (iSW signal)

This allows to go from a 3x2pt photometric analysis, to a 7x2pt data vector.
Sampling on the sum of the neutrino mass, we compare the log posterior in both cases.

We compare the spectra to the CAMB ones, in a linear scenario, without intrinsic alignement nor redshitf space dsitortions, nor magnification bias, and assuming a galaxy bias constant in each redshift bin. 

We compare the CLOE and CAMB spectra in numbers of sigma, assuming the gaussian covariance of the cross correlations with Simons Observatory. To download the SO noise curves, run the command
`git submodule update --init --recursive`
from the main CLOE directory.

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import os, sys
from os.path import join as opj

likelihood_path = os.path.realpath(os.path.join(os.getcwd(),'..'))
sys.path.insert(0, likelihood_path)
print('Setting as working directory: ', likelihood_path)

import cloe
from cloe.cobaya_interface import EuclidLikelihood
from cloe.photometric_survey.photo import Photo
from cloe.cmbx_p.cmbx import CMBX

from cobaya.model import get_model
import time
import copy

import camb



## Setup CLOE

In [ ]:
nlflag=0
barflag=0
# Set this to zero for linear CMB lensing power spectrum from CAMB (inacurate for L>1000)
# Useful for comparison with CLOE
nlflag_camb=0

info = {
'params': {
        'ombh2': 0.022445, #Omega density of baryons times the reduced Hubble parameter squared
        'omch2': 0.1205579307, #Omega density of cold dark matter times the reduced Hubble parameter squared
        'H0': 67.0, #Hubble parameter evaluated today (z=0) in km/s/Mpc
        'tau': 0.0925, #optical depth
        'mnu': 0.06, #  sum of the mass of neutrinos in eV
        'nnu': 3.046, #N_eff of relativistic species 
        'As': 2.12605e-9, #Amplitude of the primordial scalar power spectrum
        'ns': 0.96, # primordial power spectrum tilt 
        'w': -1.0, #Dark energy fluid model
        'wa': 0.0, #Dark energy fluid model
        'omk': 0.0, #curvature density
        'omegam': None, #DERIVED parameter: Omega matter density
        'omegab': None, #DERIVED parameter: Omega baryon density
        'omeganu': None, #DERIVED parameter: Omega neutrino density
        'omnuh2': None, #DERIVED parameter: Omega neutrino density times de reduced Hubble parameter squared
        'omegac': None, #DERIVED parameter: Omega cold dark matter density
        'N_eff': None},
    'theory': {'camb': 
               {'stop_at_error': True, 
                'extra_args':{'num_massive_neutrinos': 1,
                              'dark_energy_model': 'ppf', 
                             'lens_potential_accuracy': nlflag_camb}}},
    'sampler': {'evaluate': None},  
    'output': 'chains/my_euclid_experiment',
    'debug': False,
    'force': True,
    }

fid_cosmo = copy.deepcopy(info['params'])
# If you want to sample on some parameters, here with a uniform prior:
info['params']['mnu'] = {"prior": {"min": 0., "max": 1.}}
# info['params']['ns'] = {'prior':{'min':0.8, 'max':1.2}} 
# info['params']['As'] = {"prior": {"min": 2.10e-9, "max": 2.15e-9}}

In [ ]:
info['likelihood'] = {'Euclid': 
                     {'external': EuclidLikelihood, # Likelihood Class to be read as external
                     # Note: everything down below will overwrite the information read
                     # in the config folder
                     #
                     # Select which observables want to use during the analysis
                     # by setting them to True or False
                     'observables_selection': {
                         'WL': {'WL': True, 'GCphot': True, 'GCspectro': False},
                         'GCphot': {'GCphot': True, 'GCspectro': False},
                         'GCspectro': {'GCspectro': False}, 'CG': {'CG': False},
                         'add_phot_RSD': False, 'matrix_transform_phot': False
                     },
                     # Plot the selected observables matrx
                     'plot_observables_selection': False,  
                     # Switch to allow for matrix transformations of theory and data vectors
                     'matrix_transform_phot' : False, # 'BNT' or 'BNT-test,
                      # Nonlinear flags
                      # With this, the user can specify which nonlinear model they want
                      # For the time-being the available options are: 
                      # NL_flag_phot_matter
                        # 0 -> linear-only
                        # 1 -> Takahashi
                        # 2 -> Mead2016 (includes baryon corrections)
                        # 3 -> Mead2020 (w/o baryon corrections)
                        # 4 -> Mead2020_feedback (includes baryon corrections)
                        # 5 -> EE2
                        # 6 -> Bacco (matter)
                     'NL_flag_phot_matter': nlflag,
                      # NL_flag_spectro
                        # 0 -> linear-only
                        # 1 -> EFT
                     'NL_flag_spectro': 0,
                      # Baryon flag
                      # With this, the user can specify which baryon model they want
                      # For the time-being the available options are: 
                            #0 -> No baryonic feedback
                            #1 -> Mead2016 (baryons)
                            #2 -> Mead2020_feedback
                            #3 -> BCemu baryons
                            #4 -> Bacco baryons
                     'NL_flag_phot_baryon': barflag,
                      # Intrinsic alignment flag
                         #     0 -> NLA
                         #     1 -> TATT
                     'IA_flag': 0,
                      # IR-resummation method
                         #     'DST' -> Discrete sine transform
                         #     'EH' -> Eisenstein-Hu
                     'IR_resum': 'DST',
                    #Set bias model to 2 for galaxy bias constant in z bin, to compare with CAMB
                    'bias_model':2, 
                     # This flag sets the redshift evolution for baryonic parameters for emulators
                     # The options are:
                            # True -> use X(z) = X_0 * (1+z)^(-nu_X), no. of params: 7*2 = 14
                            # False -> use X_i at each redshift bin i and interpolate, no. of params: 7*10 = 70
                     'Baryon_redshift_model': False,
                     'solver': 'camb',
                     'params': {
                                # Photometric bias parameters
                                'b1_photo_bin1': 1.0997727037892875,
                                'b1_photo_bin2': 1.220245876862528,
                                'b1_photo_bin3': 1.2723993083933989,
                                'b1_photo_bin4': 1.316624471897739,
                                'b1_photo_bin5': 1.35812370570578,
                                'b1_photo_bin6': 1.3998214171814918,
                                'b1_photo_bin7': 1.4446452851824907,
                                'b1_photo_bin8': 1.4964959071110084,
                                'b1_photo_bin9': 1.5652475842498528,
                                'b1_photo_bin10': 1.7429859437184225,
                                # Magnification bias parameters
                                'magnification_bias_1': 0.0,
                                'magnification_bias_2': 0.0,
                                'magnification_bias_3': 0.0,
                                'magnification_bias_4': 0.0,
                                'magnification_bias_5': 0.0,
                                'magnification_bias_6': 0.0,
                                'magnification_bias_7': 0.0,
                                'magnification_bias_8': 0.0,
                                'magnification_bias_9': 0.0,
                                'magnification_bias_10': 0.0,
                                # Shear calibration multiplicative bias parameters                                 
                                'multiplicative_bias_1': 0.0,
                                'multiplicative_bias_2': 0.0,
                                'multiplicative_bias_3': 0.0,
                                'multiplicative_bias_4': 0.0,
                                'multiplicative_bias_5': 0.0,
                                'multiplicative_bias_6': 0.0,
                                'multiplicative_bias_7': 0.0,
                                'multiplicative_bias_8': 0.0,
                                'multiplicative_bias_9': 0.0,
                                'multiplicative_bias_10': 0.0,
                                # Spectroscopic bias parameters
                                'b1_spectro_bin1': 1.46,
                                'b1_spectro_bin2': 1.61,
                                'b1_spectro_bin3': 1.75,
                                'b1_spectro_bin4': 1.90,
                                # Intrinsic alignment parameters
                                 'a1_ia': 0, #1.72,
                                 'a2_ia': 0, #2,
                                 'b1_ia': 0, #1,
                                 'eta1_ia': 0, #-0.41,
                                 'eta2_ia': 0, #1,
                                 'beta1_ia': 0.0,
                                 'pivot_redshift': 0.,
                                # Redshift distributions nuisance parameters: shifts
                                'dz_1_GCphot': 0.0, 'dz_1_WL': 0.0,
                                'dz_2_GCphot': 0.0, 'dz_2_WL': 0.0,
                                'dz_3_GCphot': 0.0, 'dz_3_WL': 0.0,
                                'dz_4_GCphot': 0.0, 'dz_4_WL': 0.0,
                                'dz_5_GCphot': 0.0, 'dz_5_WL': 0.0,
                                'dz_6_GCphot': 0.0, 'dz_6_WL': 0.0,
                                'dz_7_GCphot': 0.0, 'dz_7_WL': 0.0,
                                'dz_8_GCphot': 0.0, 'dz_8_WL': 0.0,
                                'dz_9_GCphot': 0.0, 'dz_9_WL': 0.0,
                                'dz_10_GCphot': 0.0, 'dz_10_WL': 0.0,
                                'gamma_MG': 0.55,
                                'sigma_z': 0.002}, 
                     # k values for extrapolation of the matter power spectrum and size k-array
                     'k_max_extrap': 500.0,
                     'k_min_extrap': 1E-5,   
                     'k_samp': 1000,
                     # z limit values and size z-array
                     'z_min': 0.0,
                     'z_max': 4.0,
                     'z_samp': 100,
                     # Add RSD to photometric probes
                     'add_phot_RSD': False,
                     # Use MG gamma
                     'use_gamma_MG': False,
                     # Use redshift-dependent purity for GCspectro or not
                     'f_out_z_dep': False,
                     # Add spectroscopic redshift errors
                     'print_theory' : False,
                     # Add spectroscopic redshift errors
                     'GCsp_z_err' : True,
                     #'data': This give specifications for the paths of the input data files
                     'data': { 
                        #'sample' specifies the first folder below the main data folder
                        'sample': 'ExternalBenchmark',
                        #'spectro' and 'photo' specify paths to data files.
                        'spectro': {
                            # GC Spectro root name should contain z{:s} string
                            # to enable iteration over bins
                            'root': 'cov_power_galaxies_dk0p004_z{:s}.fits',
                            'redshifts': ["1.", "1.2", "1.4", "1.65"],
                            'edges': [0.9, 1.1, 1.3, 1.5, 1.8],
                            'scale_cuts_fourier': 'GCspectro-Fourier.yaml',
                            'Fourier': True},
                        'photo': {
                            'redshifts': [0.2095, 0.489, 0.619, 0.7335, 0.8445, 0.9595, 1.087, 1.2395, 1.45, 2.038],
                            'ndens_GC': 'niTab-EP10-RB00.dat',
                            'ndens_WL': 'niTab-EP10-RB00.dat',
                            'luminosity_ratio': 'luminosity_ratio.dat',
                            # Photometric root names should contain z{:s} string
                            # to specify IA model
                            'root_GC': 'Cls_{:s}_PosPos.dat',
                            'root_WL': 'Cls_{:s}_ShearShear.dat',
                            'root_XC': 'Cls_{:s}_PosShear.dat',
                            'root_mixing_matrix': 'fs2_mms_10zbins_32ellbins.fits',
                            'IA_model': 'zNLA',
                            # Photometric covariances root names should contain z{:s} string
                            # to specify how the covariance was calculated
                            'cov_GC': 'CovMat-PosPos-{:s}-20Bins.npz',
                            'cov_WL': 'CovMat-ShearShear-{:s}-20Bins.npz',
                            'cov_3x2pt': 'CovMat-3x2pt-{:s}-20Bins.npz',
                            'cov_model': 'Gauss',
                            'photo_data': 'standard',
                            'Fourier': True},  # or 'BNT-Gauss' if BNT selected above
                        'cmbx': {
                              'root_CMBlens': 'Cls_kCMB.dat',
                              'root_CMBlensxWL': 'Cls_kCMBxWL.dat',
                              'root_CMBlensxGC': 'Cls_kCMBxGC.dat',
                              'root_CMBisw': 'Cls_{:s}_ISWxGC.dat',
                              'ISW_model': 'zNLA',
                              'cov_7x2pt': 'Cov_7x2pt_WL_GC_CMBX.npy',}}, 
                            }} 

## Including CMB probes 

In [ ]:
info_cmbx = copy.deepcopy(info)
info_cmbx['likelihood']['Euclid']['observables_selection']= {
                    'WL': {'WL': True, 'GCphot': True, 'GCspectro': False},
                     'GCphot': {'GCphot': True, 'GCspectro': False},
                    'GCspectro': {'GCspectro': False},
                    'CMBlens': {'CMBlens':True, 'WL':True, 'GCphot':True},
                    'CMBisw': {'GCphot':True}, 'CG': {'CG': False}, 
                    'add_phot_RSD': False, 'matrix_transform_phot': False
                     }

model_cmbx = get_model(info_cmbx)

# Plot the CMBX spectra

## Intialize the likelihood in the fiducial cosmology 

In [ ]:
logposterior_cmbx = model_cmbx.logposterior([fid_cosmo['mnu']])
like = model_cmbx.likelihood['Euclid']

photo = Photo(like.cosmo.cosmo_dic, like.likefinal.data_ins.nz_dict_WL, like.likefinal.data_ins.nz_dict_GC_Phot)

cmbx = CMBX(photo)
cmbx.cmbx_update(photo)

In [ ]:
def pp2kk(ells):
    return (ells*(ells+1))**2 /4

# ells = np.unique(np.logspace(1, 3, num=100).astype('int'))
lmin = 10
lmax = 1500
# ells = np.geomspace(lmin, lmax, num=100)
ells = np.arange(lmax+1)

## CMB lensing auto power spectrum

In [ ]:
cl_kk = cmbx.Cl_kCMB(ells)
plt.loglog(ells, cl_kk)
plt.xlabel('$L$')
plt.ylabel('$C_L^{\kappa \kappa}$')

## Cross correlation of CMB lensing and galaxy clustering 

In [ ]:
ntomowl = photo.nz_WL.get_num_tomographic_bins()
ntomogc = photo.nz_GC.get_num_tomographic_bins()

cl_kgi = np.zeros([ntomogc, len(ells)])
cl_kwi = np.zeros([ntomowl, len(ells)])

for ibin in range(ntomogc):
    cl_kgi[ibin] = cmbx.Cl_kCMB_X_GC_phot(ells, ibin+1)
        
        
for ibin in range(ntomowl):
    cl_kwi[ibin] = cmbx.Cl_kCMB_X_WL(ells, ibin+1)

In [ ]:
plt.figure()

for ibin in range(ntomogc):
    plt.loglog(ells, cl_kgi[ibin], label='Bin {}'.format(ibin+1))
plt.legend(ncol=2)

plt.xlabel('$L$')
plt.ylabel('$C_L^{\kappa, \mathrm{GC photo}}$')

## Cross correlation of CMB lensing and galaxy weak lensing 

In [ ]:
plt.figure()

for ibin in range(ntomowl):
    plt.loglog(ells, cl_kwi[ibin], label='Bin {}'.format(ibin+1))
plt.legend(ncol=2)

plt.xlabel('$L$')
plt.ylabel('$C_L^{\kappa, \mathrm{WL}}$')

## Cross correlation of CMB temeperature and galaxy clustering (iSW)

In [ ]:
lmax_isw = 200
ells_isw = np.arange(lmax_isw+1)

cl_tgi = np.zeros([ntomogc, len(ells_isw)])
for ibin in range(ntomogc):
    cl_tgi[ibin] = cmbx.Cl_ISWxGC(ells_isw, ibin+1)

def fact_ell(ell):
    return ell*(ell+1)/(2.*np.pi)

In [ ]:
plt.figure()
for ibin in range(ntomogc):
    plt.plot(ells_isw, fact_ell(ells_isw)*cl_tgi[ibin], label='Bin {}'.format(ibin+1))
plt.legend(ncol=2)
plt.xlim(1, lmax_isw)

plt.xlabel('$\ell$')
plt.ylabel('$C_\ell^{T, \mathrm{GC photo}}$')


# Additional comparisons with CAMB

We compare the relative difference between CLOE and CAMB in the linear regime, assuming cross correlations with the Simons Observatory survey, assuming 36% sky fraction (i.e. total overlap of Euclid and SO). 

The validation criterion is that the absolute difference between CLOE and CAMB spectra is below 0.1 sigma.

In [ ]:
import camb
from camb import model as model_nl
from camb.sources import GaussianSourceWindow, SplinedSourceWindow

In [ ]:
pars = camb.CAMBparams()

pars.set_cosmology(H0=fid_cosmo['H0'], ombh2=fid_cosmo['ombh2'], omch2=fid_cosmo['omch2'], mnu=fid_cosmo['mnu'], omk=0, tau=fid_cosmo['tau'])
pars.InitPower.set_params(As=fid_cosmo['As'], ns=fid_cosmo['ns'], r=0)

pars.Want_CMB = True
pars.NonLinear = model_nl.NonLinear_none
pars.WantTransfer = True

zarr = np.linspace(0, 4)
pars.transfer_redshifts = zarr
_ = pars.set_matter_power(redshifts=zarr, kmax=100)

In [ ]:
sources = []
nuisance_params = like.cosmo.cosmo_dic['nuisance_parameters']
zz = np.linspace(0, 5, num=1000)
for ibin in range(ntomogc):
    sources.append(
        SplinedSourceWindow(source_type='counts', bias=nuisance_params[f'b1_photo_bin{ibin+1}'], z=zz, W=like.likefinal.data_ins.nz_dict_GC_Phot[f'n{ibin+1}'](zz))
    )
    
for ibin in range(ntomowl):
    sources.append(
        SplinedSourceWindow(source_type='lensing', bias=nuisance_params[f'b1_photo_bin{ibin+1}'], z=zz, W=like.likefinal.data_ins.nz_dict_GC_Phot[f'n{ibin+1}'](zz))
    )

In [ ]:
pars.SourceWindows = sources
pars.SourceTerms.limber_windows = True
pars.SourceTerms.limber_phi_lmin = 1
pars.SourceTerms.limber_windows = True

pars.SourceTerms.counts_density = True
pars.SourceTerms.counts_lensing = False # magnification bias
pars.SourceTerms.counts_velocity = False
pars.SourceTerms.counts_radial = False
pars.SourceTerms.counts_redshift = False # redshift distortions
pars.SourceTerms.counts_potential = False
pars.SourceTerms.counts_timedelay = False
pars.SourceTerms.counts_evolve = False
pars.SourceTerms.counts_ISW = False

# lmax_camb = 3000
pars.set_for_lmax(3000, lens_potential_accuracy=0)

camb_results = camb.get_results(pars)
clscambsources = camb_results.get_source_cls_dict(raw_cl=True)

camb_cls_pp = camb_results.get_lens_potential_cls(lmax=lmax, raw_cl = True)[:, 0] 

## Define covariances and noise models for power spectra


In [ ]:
# Get Gaussian covariance estimates

def get_gauss_autosigma(cl, nl, lmax=3000, fsky=0.36, return_sigma=True):
    ls = np.arange(lmax+1)
    cov = 1./(2*ls+1)/fsky * 2*(cl[:lmax+1]+nl[:lmax+1])**2
    if return_sigma:
        return np.sqrt(cov)
    else:
        return cov

def get_gauss_crosssigma(cla, nla, clb, nlb, clab, lmax=3000, fsky=0.36, return_sigma=True):
    ls = np.arange(lmax+1)
    cov = 1./(2*ls+1)/fsky * ((cla[:lmax+1]+nla[:lmax+1])*(clb[:lmax+1]+nlb[:lmax+1]) + clab**2)
    if return_sigma:
        return np.sqrt(cov)
    else:
        return cov


To download the SO noise curves, run the command
`git submodule update --init --recursive`
from the main CLOE directory.

In [ ]:
# Get CMB lensing N0 bias
# We load the noise curves generated by SO collaboration 
# We take the Minimum Variance baseline iterative analysis 
# (recommanded science case in their README)
savepath = opj(os.path.dirname(os.path.dirname(cloe.__file__)), 'data', 'ExternalBenchmark')

SO_lensing = opj(savepath, 'so_noise_models', 'LAT_lensing_noise/lensing_v3_1_1/nlkk_v3_1_0_deproj0_SENS1_fsky0p4_it_lT30-3000_lP30-5000.dat')
N0s_SO = np.loadtxt(SO_lensing).T
N0_kCMB = N0s_SO[7]
# SO noise curve start at ell = 2, so we append ell = 0 and ell = 1 to get more easily N0(ell)
N0_kCMB = np.insert(N0_kCMB, 0, 0)
N0_kCMB = np.insert(N0_kCMB, 0, 0)

# TT noise and power spectrum

sigma_T = [33.]
beam = 7.
theta2 = (beam*np.array([np.pi/60./180.]))**2/8.0/np.log(2.0)
sigma_T *= np.array([np.pi/60./180.])
nl_TT = sigma_T**2*np.exp(ells*(ells+1)*theta2)
cl_TT = np.array([like.cosmo.cosmo_dic['Cl']['tt'][int(ell)] for ell in ells])

# We load here the temperature noise curve from SO
SO_temperature = opj(savepath, 'so_noise_models', 'LAT_comp_sep_noise/v3.1.0/SO_LAT_Nell_T_atmv1_baseline_fsky0p4_ILC_CMB.txt')
NT_SO = np.loadtxt(SO_temperature).T
ind_SO = np.where(ells>=40)[0]
nl_TT[ind_SO] = NT_SO[1,ells[ind_SO].astype(int)-40]

In [ ]:

# Define here the galaxies number density in each bin (in arcmin-2 converted to steradians)
ngal = 3*3600*(180/np.pi)**2
# Galaxy shot noise
GG_noise = 1./ngal *np.ones(lmax+1)

# Define here the total intrinsic ellipticity dispersion
sig_eps = 0.3
# Weaklensing shape noise
LL_noise = sig_eps**2/ngal / 2. * np.ones(lmax+1)


## Comparing CMB lensing auto correlation

In [ ]:
ls = np.arange(lmin, lmax+1)

In [ ]:
plt.loglog(ls, cl_kk[ls], label='CLOE')
plt.loglog(ls, pp2kk(ls)*camb_cls_pp[ls], label='CAMB', ls='--')

plt.legend()
plt.xlabel('$L$')
plt.ylabel('$C_L^{\kappa \kappa}$')

In [ ]:
sigma = get_gauss_autosigma(cl_kk, N0_kCMB, lmax=lmax)
plt.loglog(ls, np.abs(camb_cls_pp[ls]*pp2kk(ls) - cl_kk[ls])/sigma[ls])
plt.ylabel(r'd(CLOE, CAMB) $[\sigma]$', fontsize=15)
plt.xlabel('$L$', fontsize=15)

plt.title('$C_L^{\kappa, \kappa}$')

plt.axhline(y=0.1, c='k', ls=':')


## Comparing the CMB lensing cross correlations

In [ ]:
# Note from CAMB DEMO notebook 
# Note that P is CMB lensing, as a deflection angle power (i.e. PxP is [l(l+1)]^2C_l\phi\phi/2\pi)

In [ ]:
# Galaxy clustering auto correlations

cl_gigi = np.zeros([ntomogc, lmax+1])

for ibin in range(ntomogc):
    cl_gigi[ibin] = photo.Cl_GC_phot(np.arange(lmax+1), ibin+1, ibin+1)


In [ ]:
# Weak lensing auto correlations

cl_wiwi = np.zeros([ntomowl, lmax+1])

for ibin in range(ntomowl):
    cl_wiwi[ibin] = photo.Cl_WL(np.arange(lmax+1), ibin+1, ibin+1)

In [ ]:
for ibin in range(ntomogc):
    plt.plot(ls, cl_kgi[ibin][ls], label='CLOE bin {}'.format(ibin+1))
    plt.plot(ls, clscambsources[(f'PxW{ibin+1}')][ls]*np.sqrt(pp2kk(ls)), label='CAMB bin {}'.format(ibin+1), ls='--')
    
plt.loglog()
plt.legend(ncol=2)
plt.xlabel('$L$')
plt.ylabel('$C_L^{\kappa, \mathrm{GC photo}}$')
plt.title('Cross correlation of CMB lensing with galaxy density photo')

In [ ]:
for ibin in range(ntomogc):
    sigma = get_gauss_crosssigma(cl_kk, N0_kCMB, cl_gigi[ibin], GG_noise, cl_kgi[ibin], lmax=lmax)
    plt.plot(ls,  np.abs(clscambsources[(f'PxW{ibin+1}')][ls]*np.sqrt(pp2kk(ls)) - cl_kgi[ibin][ls])/sigma[ls], label='bin {}'.format(ibin+1))

plt.loglog()
plt.axhline(y=0.1, c='k', ls=':')

plt.legend(ncol=2)
plt.xlabel('$L$', fontsize=15)
plt.ylabel('d(CLOE, CAMB) $[\sigma]$', fontsize=15)

plt.axhline(y=0, c='k')

plt.title('$C_L^{\kappa, \mathrm{GC photo}}$')

In [ ]:
for ibin in range(ntomogc):
    plt.plot(ls, cl_kwi[ibin][ls], label='CLOE bin {}'.format(ibin+1))
    plt.plot(ls, clscambsources[(f'PxW{ibin+11}')][ls]*np.sqrt(pp2kk(ls)), label='CAMB bin {}'.format(ibin+1), ls='--')
    
plt.loglog()

plt.legend(ncol=2)
plt.xlabel('$\ell$', fontsize=15)
plt.ylabel('$C_\ell^{\kappa, \mathrm{L}}$', fontsize=15)
plt.title('Cross correlation of CMB lensing with galaxy weak lensing')

In [ ]:
for ibin in range(ntomogc):
    sigma = get_gauss_crosssigma(cl_kk, N0_kCMB, cl_wiwi[ibin], LL_noise, cl_kwi[ibin], lmax=lmax)
    plt.plot(ls,  np.abs(clscambsources[(f'PxW{ibin+11}')][ls]*np.sqrt(pp2kk(ls)) - cl_kwi[ibin][ls])/sigma[ls], label='bin {}'.format(ibin+1))

plt.loglog()
plt.axhline(y=0.1, c='k', ls=':')

plt.legend(ncol=2)
plt.xlabel('$\ell$', fontsize=15)
plt.ylabel('d(CLOE, CAMB) $[\sigma]$', fontsize=15)
plt.title('$C_\ell^{\kappa, \mathrm{L}}$')
plt.axhline(y=0, c='k')

## Comparing iSW 

In [ ]:
clscamb = camb_results.get_cmb_unlensed_scalar_array_dict(CMB_unit=None, raw_cl=True)
clscmb = camb_results.get_cmb_power_spectra(CMB_unit=None, raw_cl=True)
cltt = clscmb['total'].T[0]
cmb_muk = 2.7255 *1e6

In [ ]:
plt.figure()

for ibin in range(ntomogc):
    plt.plot(ells_isw, fact_ell(ells_isw)*cl_tgi[ibin], label='CLOE Bin {}'.format(ibin+1))
    plt.plot(ells_isw, fact_ell(ells_isw)*clscamb[f'TxW{ibin+1}'][ells_isw], label='CAMB Bin {}'.format(ibin+1), ls='--')

    
plt.legend(ncol=2)

plt.xlabel('$\ell$', fontsize=15)
plt.ylabel('$\ell(\ell+1)/2\pi \;  C_\ell^{T, \mathrm{GC photo}}$', fontsize=15)


In [ ]:
plt.figure()

for ibin in range(ntomogc):
    sigma = get_gauss_crosssigma(cltt, nl_TT/cmb_muk**2, cl_gigi[ibin], GG_noise, cl_tgi[ibin], lmax=lmax_isw)
    plt.plot(ells_isw, np.abs(cl_tgi[ibin][ells_isw] -  clscamb[f'TxW{ibin+1}'][ells_isw])/sigma[ells_isw], label='Bin {}'.format(ibin+1))

plt.legend(ncol=2)
plt.loglog()
plt.axhline(y=0.1, c='k', ls=':')
plt.xlim(2, lmax_isw)
plt.xlabel('$\ell$', fontsize=15)
plt.title('iSW, $C_\ell^{T, \mathrm{GC photo}} $', fontsize=15)
plt.ylabel('d(CLOE, CAMB), $[\sigma]$')

# Sampling the posterior with the sun of the neutrino mass parameter

We update likleihood model parameters to match the values of the mock data vector stored on disk.

We sample only the sum of the neutrino masses, leaving all other parameters fixed. 

In [ ]:
info['likelihood']['Euclid']['params']['a1_ia'] = 1.72
info['likelihood']['Euclid']['params']['a2_ia'] = 2
info['likelihood']['Euclid']['params']['b1_ia'] = 1.
info['likelihood']['Euclid']['params']['eta1_ia'] = -0.41
info['likelihood']['Euclid']['params']['eta2_ia'] = 1.
info['likelihood']['Euclid']['bias_model'] = 1.

info_cmbx['likelihood']['Euclid']['params']['a1_ia'] = 1.72
info_cmbx['likelihood']['Euclid']['params']['a2_ia'] = 2
info_cmbx['likelihood']['Euclid']['params']['b1_ia'] = 1.
info_cmbx['likelihood']['Euclid']['params']['eta1_ia'] = -0.41
info_cmbx['likelihood']['Euclid']['params']['eta2_ia'] = 1.
info_cmbx['likelihood']['Euclid']['bias_model'] = 1.


In [ ]:
model = get_model(info)
model_cmbx = get_model(info_cmbx)


In [ ]:
mnu = np.linspace(0.056, 0.064, num=8)
log = np.zeros_like(mnu)
log_cmbx = np.zeros_like(mnu)
for i, m in enumerate(mnu):
    log[i] = model.logposterior([m]).loglikes[0]
    log_cmbx[i] = model_cmbx.logposterior([m]).loglikes[0]
    print(i, m, log[i], log_cmbx[i])

In [ ]:
plt.plot(mnu, np.exp(log), label="3x2pt")
plt.plot(mnu, np.exp(log_cmbx), label="7x2pt")
plt.axvline(x=0.06, c='r', label="fiducial")
plt.legend()
plt.xlabel(r"$\sum m_\nu$")
plt.ylabel("log posterior")